In [ ]:
%%bash

# Test Run with 10 designs 
target="PDL1.json"
target_name="${target%.json}"  # Strip the .json extension
target_path="/workspace/settings_target/$target"
log_dir="/workspace/outputs/$target_name"
log_file="$log_dir/${target_name}-bindcraft_log.txt"

# Ensure log directory exists
mkdir -p "$log_dir"

echo "[INFO] Checking target settings file..."
if [ ! -f "$target_path" ]; then
  echo "[ERROR] Target file '$target' not found in /workspace/settings_target/"
  exit 1
fi

echo "[INFO] Starting BindCraft with target '$target'"
echo "[INFO] Logging output to $log_file"

nohup python /app/bindcraft/bindcraft.py \
  --settings "$target_path" \
  --filters /workspace/settings_filters/default_filters.json \
  --advanced /workspace/settings_advanced/default_4stage_multimer.json \
  > "$log_file" 2>&1 &

echo "[INFO] BindCraft is now running in the background (PID=$!)"

In [ ]:
!tail -n 50 -f /workspace/outputs/${log_file}

In [1]:
import ipywidgets as widgets
from IPython.display import display
uploader = widgets.FileUpload(accept='.json', multiple=False)
display(uploader)

FileUpload(value=(), accept='.json', description='Upload')

In [ ]:
import sys
import os
DEV_DIR = '/home/yarrow/projects/bindcraft-runpod/functions/'
PROD_DIR = '/app/bindcraft/functions'

path_used = DEV_DIR if os.path.exists(DEV_DIR) else PROD_DIR
print(f"Using function path: {path_used}")
sys.path.append(path_used)

from main_UI import launch_all_ui

launch_all_ui()


Using function path: /home/yarrow/projects/bindcraft-runpod/functions/
Step 1: Edit your target JSON file.


Text(value='PDL1.json', description='File Name:', layout=Layout(width='50%'))

Text(value='PDL1', description='Subfolder to save json file to (optional):', style=TextStyle(description_width…

Text(value='/workspace/outputs/PDL1', description='design_path:', style=TextStyle(description_width='initial')…

Text(value='PDL1-Binder', description='binder_name:', style=TextStyle(description_width='initial'))

...

Button(button_style='primary', description='Run BindCraft', layout=Layout(width='30%'), style=ButtonStyle())

Output()

Optional: Upload a PDB or JSON file to your working directory.


FileUpload(value=(), accept='.json,.pdb', description='Upload JSON/PDB')

TypeError: tuple indices must be integers or slices, not Bunch